In [85]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint
import json
from ipywidgets.embed import embed_minimal_html


# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


In [86]:
wine_location = pd.read_csv("../Output/wine_cleaned.csv")
wine_location.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [87]:
#Add the latitude  and longitude  empty fields to prime for geocode info
wine_location['latitude'] = ""
wine_location['longitude'] = ""
#wine_location['region_latitude'] = ""
#wine_location['region_longitude'] = ""
#wine_location.head()

wine_location_short = wine_location[["country", "province","latitude", "longitude"]]
#wine_grouping = wine_location_short.groupby(["province", "country"])



wine_grouping = pd.DataFrame(wine_location_short)

wine_grouping.drop_duplicates(subset='province', keep='first', inplace=True)

wine_dedup = pd.DataFrame(wine_grouping)

#wine_grouping["country", "province","latitude", "longitude"].unique()
wine_dedup.head()





,country,province,latitude,longitude
0,Italy,Sicily & Sardinia,,
1,Portugal,Douro,,
2,US,Oregon,,
3,US,Michigan,,
5,Spain,Northern Spain,,


In [88]:
# Output File (CSV) testing duplicate removal
#output_data_file = "../Output/duplicate_review.csv"
#wine_dedup.to_csv(output_data_file, index=False, header=True)

In [89]:
wine_dedup.dtypes

country      object
province     object
latitude     object
longitude    object
dtype: object

In [150]:
#Get the lat and long for the API

# Build the endpoint URL
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'

#Need the province , country from the table and combine together
#Create loop to gather the information

print("Beginning Data Retrieval")
print("-----------------------------")
set=1
# Loop through the list of province & country and perform a request for data on each
for index, row in wine_dedup.iterrows():
    
    #Get the fields from the table
    province_name = row['province']
    country_name = row['country']

    location = (province_name) + ", " + country_name
    #print(combo)
    geo_response = requests.get(geo_url.format(location, g_key)).json()
    #                            & province_name & ", " & country & "&key=" g_key).json()
    #print(geo_response)
    #print(json.dumps(geo_response, indent=4, sort_keys=True))
    try:
        # Extract latitude and longitude
        
        #print(geo_response["results"][0]["geometry"]["location"]["lat"])
        #print(geo_response["results"][0]["geometry"]["location"]["lng"])

        wine_dedup.loc[index, "latitude"] = geo_response["results"][0]["geometry"]["location"]["lat"]
        wine_dedup.loc[index, "longitude"] = geo_response["results"][0]["geometry"]["location"]["lng"]
        
        if index % 100 == 0:
            print(f"Processing Record {index} of location {province_name}, {country_name}.")
        
    except (KeyError):
        print(f"{province_name}, {country_name} not found for Record {index} . Skipping...")
        set+= 1
    except (IndexError):
        print(f"Location not found has Index error {index} for {province_name}, {country_name}. Skipping...")
        set+= 1




Beginning Data Retrieval
-----------------------------
Processing Record 0 of location Sicily & Sardinia, Italy.
Location not found has Index error 16 for Other, Argentina. Skipping...
Location not found has Index error 208 for Coastal Region, South Africa. Skipping...
Processing Record 400 of location Italy Other, Italy.
Location not found has Index error 1186 for Thrace, Turkey. Skipping...
Location not found has Index error 1440 for Wagram-Donauland, Austria. Skipping...
Location not found has Index error 2383 for Terras do Dão, Portugal. Skipping...
Processing Record 2500 of location Panciu, Romania.
Location not found has Index error 5082 for Pocerina, Serbia. Skipping...
Location not found has Index error 7586 for Pageon, Greece. Skipping...
Location not found has Index error 7668 for Vinho Espumante de Qualidade, Portugal. Skipping...
Processing Record 9500 of location Sebes, Romania.
Location not found has Index error 14164 for Guerrouane, Morocco. Skipping...
Location not foun

In [151]:
wine_dedup.head()

,country,province,latitude,longitude
0,Italy,Sicily & Sardinia,37.6,14.0154
1,Portugal,Douro,41.5013,-5.51229
2,US,Oregon,43.8041,-120.554
3,US,Michigan,44.3148,-85.6024
5,Spain,Northern Spain,43.3366,-3.20272


In [153]:
# Combine the data into a single dataset

#wine_location.drop(['latitude', 'longitude'], axis=1, inplace=True) 
                 
#wine_location.head()
wine_location_complete = pd.merge(wine_location, wine_dedup,  how='left', 
                                  left_on=['province','country'], right_on = ['province','country'])

wine_location_complete.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year,latitude,longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,37.6,14.0154
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,41.5013,-5.51229
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,43.8041,-120.554
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,44.3148,-85.6024
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,43.8041,-120.554


In [154]:
# Output File (CSV)
output_data_file = "../Output/wine_cleaned_location.csv"
wine_location_complete.to_csv(output_data_file, index=False, header=True)

In [222]:
###Remove data when the lat and long data is blank
wine_location_complete = wine_location_complete.dropna(subset=['latitude', 'longitude'])
# Fill NaN values and convert to float
#wine_location_complete = wine_location_complete.astype({'latitude': 'float64', 'latitude': 'float64'}).fillna(0)


#Another way to change the fields to numeric other than the above
wine_location_complete[["latitude", "longitude"]] = wine_location_complete[["latitude", "longitude"]].apply(pd.to_numeric)

wine_location_complete.head()

##Color change for sizing of the points

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year,latitude,longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,37.599994,14.015356
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,41.501277,-5.512293
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,43.804133,-120.554201
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,44.314844,-85.602364
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,43.804133,-120.554201


In [223]:
wine_location_complete.dtypes

country                   object
description               object
points                     int64
price                    float64
province                  object
region_1                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
Wine_Year                  int64
latitude                 float64
longitude                float64
dtype: object

In [217]:
#Group the items need for the map
wine_location_complete_group1 = wine_location_complete.groupby(['province','country','latitude','longitude'])

#Get the average for points and create new dataframe
wine_location_map_test2 = wine_location_complete_group1["points"].mean().to_frame(name = 'avg_points').reset_index()

wine_location_map_test2.head(20)

,province,country,latitude,longitude,avg_points
0,Achaia,Greece,38.115873,21.952249,85.800000
1,Aconcagua Costa,Chile,-32.803550,-70.944224,87.629630
2,Aconcagua Valley,Chile,-32.803550,-70.944224,88.163793
3,Aegean,Turkey,38.000000,25.000000,88.954545
4,Ahr,Germany,50.465601,6.951237,90.222222
5,Alenquer,Portugal,39.057400,-9.006365,87.285714
6,Alentejano,Portugal,38.200381,-7.800085,87.905016
7,Alentejo,Portugal,38.200381,-7.800085,88.653846
8,Alenteo,Portugal,38.200381,-7.800085,88.000000
9,Algarve,Portugal,37.017954,-7.930834,86.888889


In [218]:
# Store latitude and longitude in locations
locations = wine_location_map_test2[["latitude", "longitude"]]

points = wine_location_map_test2["avg_points"]

province_map = wine_location_map_test2["province"]

In [ ]:
#Using this example to plot out the color coding!!
Starbucks coffee shops and KFC outlets in the UK by plotting both on the same map:

import gmaps
import gmaps.datasets

gmaps.configure(api_key='AI...')

df = gmaps.datasets.load_dataset_as_df('starbucks_kfc_uk')

starbucks_df = wine_location_map[wine_location_map['points']  'starbucks']
starbucks_df = starbucks_df[['latitude', 'longitude']]

kfc_df = df[df['chain_name'] == 'kfc']
kfc_df = kfc_df[['latitude', 'longitude']]


starbucks_layer = gmaps.symbol_layer(
    starbucks_df, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 150, 0, 0.4)', scale=2
)

kfc_layer = gmaps.symbol_layer(
    kfc_df, fill_color='rgba(200, 0, 0, 0.4)',
    stroke_color='rgba(200, 0, 0, 0.4)', scale=2
)

fig = gmaps.figure()
fig.add_layer(starbucks_layer)
fig.add_layer(kfc_layer)
fig

In [219]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")
#print(fig)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=points, #hover_text=province_map,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

gmaps.Marker

#print(heat_layer)

# Customize the size of the figure


figure_layout = {
    'width': '1500px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '12px',
    'margin': '0 auto 0 auto'
}

#1500 x 2100
fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)



# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='12px', w…

In [208]:
# Assign the marker layer to a variable
#markers = gmaps.marker_layer(locations, hover_text=province_map)
#symbol_layer = gmaps.symbol_layer(locations, hover_text=province_map)
# Add the layer to the map
#fig.add_layer(symbol_layer)
#fig

In [221]:
# Save the figure in html
embed_minimal_html('../images/WinePoints5.html', views=[fig])